In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from keras import backend, optimizers
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
#give your dataset path in the place of ...
infected_dir =r"...\cell_images\Parasitized"
uninfected_dir =r"...\cell_images\uninfected"
infected_cells = os.listdir(infected_dir)
uninfected_cells = os.listdir(uninfected_dir)

In [ ]:
print(len(infected_cells))
print(len(uninfected_cells))

In [ ]:
def plot_examples(photo_names: list, photo_dir: str) -> None:
    plt.figure(figsize = (15,15))
    for num, img_name in enumerate(photo_names):
        plt.subplot(1, len(photo_names), num + 1)
        img_path = os.path.join(photo_dir, img_name)
        pil_im = Image.open(img_path, 'r')
        plt.imshow(pil_im)
        plt.tight_layout()
    plt.show()

In [ ]:
example_infected_cells = infected_cells[:6]

plot_examples(example_infected_cells, infected_dir)

In [ ]:
example_uninfected_cells = uninfected_cells[:6]

plot_examples(example_uninfected_cells, uninfected_dir)

In [ ]:
def preprocess_images(images: list, base_path: str, img_label: int, resize: tuple = (50, 50)) -> tuple:
    X, y = [], []
    for image in images:
        try:
            img_arr = plt.imread(os.path.join(base_path, image))
            resized_img = cv2.resize(img_arr, resize)
            y.append(img_label)
            X.append(resized_img)
        except OSError as exc:
            None
    return X, y

In [ ]:
X_p, y_p = preprocess_images(infected_cells, infected_dir, 1)
X_u, y_u = preprocess_images(uninfected_cells, uninfected_dir, 0)

In [ ]:
X = np.array([*X_p, *X_u])
y = np.array([*y_p, *y_u])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

In [ ]:
train_image_generator = ImageDataGenerator(
        fill_mode='nearest',  
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=30,  
        zoom_range = 0.3, 
        width_shift_range=0.2, 
        height_shift_range=0.2,  
        horizontal_flip=True,  
        vertical_flip=False  
)  

validation_image_generator = ImageDataGenerator()

In [ ]:
train_data = train_image_generator.flow(X_train, y_train, batch_size=64, shuffle=True)
validation_data = validation_image_generator.flow(X_test, y_test, batch_size=64, shuffle=True)

In [ ]:
class CNNNet:
    @staticmethod
    def build(width: int, height: int, depth: int, classes: int):
        model = Sequential()
        input_shape = (height, width, depth)
        channel_dimension = -1
        
        if backend.image_data_format() == "channels_first":
            input_shape = (depth, height, width)
            channel_dimension = 1
            
        # first CONV => RELU => POOL => CONV => RELU layer set
        model.add(Conv2D(32, (3, 3), input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Conv2D(32, (3, 3), input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Dropout(0.2))

        # second CONV => RELU => POOL => CONV => RELU layer set
        model.add(Conv2D(32, (3, 3), input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Conv2D(32, (3, 3), input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Dropout(0.2))

        # FC => RELU layers
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Dropout(0.5))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [ ]:
height, width, depth = 50, 50, 3
classes = 2

model = CNNNet.build(width, height, depth, classes)
model.summary()

In [ ]:
steps_per_epoch = X_train.shape[0] // 64
adam_optimizer = optimizers.Adam(lr = 0.001, decay = 0.001 / 64)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_data, 
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_data,
    validation_steps=6
)

In [ ]:
def show_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('Loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('Accuracy')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [ ]:
show_history(history)

In [ ]:
predict = model.evaluate_generator(validation_data, steps=4)
print(f"Loss on test data set: {predict[0]:.5f}. Accuracy on test dataset: {predict[1] * 100:.2f}%")

In [ ]:
def show_random_predictions(model, X_test):
    predictions = model.predict(X_test)
    print(predictions)
    for i, row in enumerate(predictions):
        is_infected = row[0] <= row[1]
        label = "infected" if is_infected else "Uninfected"
        plt.title(label)
        plt.imshow(X_test[i])
        plt.show()

In [ ]:
random_data = X_test[np.random.randint(X_test.shape[0],size=10), :]
show_random_predictions(model, random_data)